In [1]:
# import
import pandas as pd
from datetime import datetime, timedelta

In [2]:
from utils import recommended_pe_ratio
from warning import rule_price_increase, rule_largest_gain, rule_accelerating_growth, rule_falling_below_ma, stock_warning_system
from earning import upcoming_earnings
from trade_decision import *
import yfinance as yf

In [3]:
def get_stock_info_on_date(tickers, short_window, long_window, rsi_buy_signal, rsi_sell_signal, rsi_window, date=datetime.today().strftime("%Y-%m-%d"), history_days=600, long_term_ma=200):
    print("Today is " + date)
    end_date = datetime.strptime(date, '%Y-%m-%d') + timedelta(days=1)  # To include the end date in the fetch
    start_date = end_date - timedelta(days=history_days)

    # market_index = yf.Ticker("^GSPC")  # S&P 500 Index
    # market_data = market_index.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    # market_signal = get_market_exit_signal(market_data, long_term_ma)
    # print(market_signal)

    data = {'Company Code': [], 'Date': [], 'Daily Price': [], 'Recommendation': [],'60 DAY RSI':[], 'P/E Ratio': [], 'Recommended PE':[], 'Category': [],'Dividend Yield': [], 'Market Cap': [], 'Earnings Growth': [], 'One Year Target': [], 'Analyst Buy': [], 'Analyst Hold': [], 'Analyst Sell': []}
    
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        hist_data = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        if hist_data.empty:
            continue  # Skip if no data for the given date range
        
        last_date = hist_data.index[-1].strftime('%Y-%m-%d')  # Last date in the historical data
        daily_price = hist_data['Close'].iloc[-1]  # Last close price
        # Apply buy/sell signal functions (as previously defined)
        risk_buy_signal, buy_data = check_buy_signal(hist_data, short_window=short_window, long_window=long_window, rsi_buy_signal=rsi_buy_signal, rsi_window=rsi_window)
        buy_signal, _ = check_buy_signal(hist_data, short_window=short_window, long_window=long_window, rsi_buy_signal=rsi_buy_signal, rsi_window=rsi_window)
        sell_signal, _ = check_sell_signal(hist_data, short_window=short_window, long_window=long_window, rsi_sell_signal=rsi_sell_signal, rsi_window=rsi_window)
        recommendation = 'BUY' if buy_signal else 'SELL' if sell_signal else 'RISK BUY' if risk_buy_signal else None

        if recommendation:
            pe_ratio = stock.info.get('trailingPE', 'N/A')
            category = stock.info.get('sector', 'N/A')
            dividend_yield = stock.info.get('dividendYield', 'N/A') * 100 if stock.info.get('dividendYield') is not None else 'N/A'
            market_cap = stock.info.get('marketCap', 'N/A')
            earnings_growth = stock.info.get('earningsGrowth', 'N/A')
            one_year_target = stock.info.get('targetMeanPrice')
            analyst_buy_ratings = stock.info.get('buyRatingCount')
            analyst_hold_ratings = stock.info.get('holdRatingCount')
            analyst_sell_ratings = stock.info.get('sellRatingCount')
            data['Company Code'].append(ticker)
            data['Recommendation'].append(recommendation)
            data['60 DAY RSI'].append(buy_data['RSI'].iloc[-1])
            data['P/E Ratio'].append(pe_ratio)
            data['Category'].append(category)
            data['Dividend Yield'].append(dividend_yield)
            data['Market Cap'].append(market_cap)
            data['Earnings Growth'].append(earnings_growth)
            data['Recommended PE'].append(recommended_pe_ratio(category))
            data['One Year Target'].append(one_year_target)
            data['Analyst Buy'].append(analyst_buy_ratings)
            data['Analyst Hold'].append(analyst_hold_ratings)
            data['Analyst Sell'].append(analyst_sell_ratings)
            data['Date'].append(last_date)
            data['Daily Price'].append(daily_price)

    return pd.DataFrame(data)


# Everyday

In [4]:
def every_day_printer(date = datetime.today().strftime("%Y-%m-%d")):
  # Example usage
  # tickers = get_nasdaq_tickers() # List of tickers
  # tickers = get_sp500_tickers()
  # date = '2024-01-06'
  crypto = ['BTC-USD', 'ETH-USD']
  cn_tickers = ['601288.SS', '688091.SS', '600221.SS', '600733.SS', '300169.SZ','603628.SS', '601808.SS']
  tickers = ['AAPL','AAL', 'ASML','AMD', 'ABNB','ALK', 'ALB', 'ALTM', 'ARKK','ARM', 'AMC','ANET','BABA','BAC','BILI','BNTX','C', 'CDNS', 'CVNA','COF', 'COIN', 'COST', 'CRM', 'DASH', 'DAL', 'DDOG', 'DIS', 'DUOL', 'GOOGL', 'HOOD', 'ISRG', 'JPM', 'KO', 'KRE', 'LYFT', 'LLY', 'MDB','META', 'MSFT', 'NKE','NEE', 'NVDA', 'NVO', 'ORCL', 'OXY', 'PANW', 'PDD', 'PLTR', 'PFE', 'RBLX', 'SHOP', 'SNOW', 'SOFI', 'SMCI', 'SNAP', 'SBUX', 'SPOT', 'SOUN', 'INTC', 'BA', 'TMF', 'TRV', 'TSLA', 'TSM', 'UAL', 'UBER','UHAL', 'UPS', 'WFC','XOM', 'DHI', 'MMM', 'NFLX']
  tickers = tickers + cn_tickers + crypto
  # tickers = sp500_tickers + nasdaq_tickers

  # File to write the results
  filename = "stock_analysis_results.txt"
  print(tickers)

  # Open the file in append mode
  with open(filename, 'a') as file:
    file.write(f"Analysis Date: {date}\n")
    file.write("Tickers:\n")
    file.write(f"{tickers}\n\n")

    stock_table = get_stock_info_on_date(tickers, date=date, short_window=SHORT_WINDOW, long_window=LONG_WINDOW, rsi_buy_signal=RSI_BUY_SIGNAL, rsi_sell_signal=RSI_SELL_SIGNAL, rsi_window=RSI_WINDOW )
    file.write("Stock Table:\n")
    file.write(stock_table.to_string() + "\n\n")  # Convert DataFrame to string before writing
    # Set the display.max_rows option to None
    pd.set_option('display.max_rows', None)
    display(stock_table)

    warning_rules = [rule_price_increase, rule_largest_gain, rule_accelerating_growth, rule_falling_below_ma]
    warnings = stock_warning_system(tickers, date, warning_rules)
    print("Warnings:", warnings)
    file.write("Warnings:\n")
    file.write(f"{warnings}\n\n")

    # Get stocks with upcoming earnings and write to file
    file.write("Stocks with Upcoming Earnings:\n")
    stocks_with_upcoming_earnings = upcoming_earnings(tickers, date)
    for ticker, earnings_date in stocks_with_upcoming_earnings:
      print(f"{ticker}: Earnings on {earnings_date}")
      file.write(f"{ticker}: Earnings on {earnings_date}\n")

    file.write(f"\n***************************************************************************************************\n\n")


In [5]:
# import pandas as pd
# import yfinance as yf
# from datetime import datetime

# # Re-initializing after code execution state reset
# # Define the start date and end date
# start_date = "2023-10-01"
# end_date = datetime.today().strftime("%Y-%m-%d")  # Today's date

# # Use a commonly traded stock symbol to get trade dates (assuming NYSE calendar for this example)
# ticker = "^GSPC"  # S&P 500 Index

# # Download historical data for the ticker
# data = yf.download(ticker, start=start_date, end=end_date)

# # Extract the dates where trades occurred
# trade_dates = data.index.strftime("%Y-%m-%d").tolist()

# trade_dates = trade_dates[-10:]
# for trade_date in trade_dates:
#   print(trade_date)
#   every_day_printer(trade_date)

In [6]:
every_day_printer(date = "2025-03-14")

['AAPL', 'AAL', 'ASML', 'AMD', 'ABNB', 'ALK', 'ALB', 'ALTM', 'ARKK', 'ARM', 'AMC', 'ANET', 'BABA', 'BAC', 'BILI', 'BNTX', 'C', 'CDNS', 'CVNA', 'COF', 'COIN', 'COST', 'CRM', 'DASH', 'DAL', 'DDOG', 'DIS', 'DUOL', 'GOOGL', 'HOOD', 'ISRG', 'JPM', 'KO', 'KRE', 'LYFT', 'LLY', 'MDB', 'META', 'MSFT', 'NKE', 'NEE', 'NVDA', 'NVO', 'ORCL', 'OXY', 'PANW', 'PDD', 'PLTR', 'PFE', 'RBLX', 'SHOP', 'SNOW', 'SOFI', 'SMCI', 'SNAP', 'SBUX', 'SPOT', 'SOUN', 'INTC', 'BA', 'TMF', 'TRV', 'TSLA', 'TSM', 'UAL', 'UBER', 'UHAL', 'UPS', 'WFC', 'XOM', 'DHI', 'MMM', 'NFLX', '601288.SS', '688091.SS', '600221.SS', '600733.SS', '300169.SZ', '603628.SS', '601808.SS', 'BTC-USD', 'ETH-USD']
Today is 2025-03-14


,Company Code,Date,Daily Price,Recommendation,60 DAY RSI,P/E Ratio,Recommended PE,Category,Dividend Yield,Market Cap,Earnings Growth,One Year Target,Analyst Buy,Analyst Hold,Analyst Sell
0,ALB,2025-03-14,75.230003,BUY,38.874573,N/A,"(N/A, N/A)",Basic Materials,215.0,9203614720,N/A,101.86667,None,None,None
1,AMC,2025-03-14,2.900000,BUY,36.666669,N/A,"(N/A, N/A)",Communication Services,N/A,1278615168,N/A,3.51250,None,None,None
2,CVNA,2025-03-14,181.179993,BUY,42.358087,116.30817,"(N/A, N/A)",Consumer Cyclical,N/A,21627006976,N/A,268.15600,None,None,None
3,ISRG,2025-03-14,484.440002,BUY,44.765719,75.772514,"(15, 25)",Healthcare,N/A,174039695360,0.118,630.35000,None,None,None
4,SOUN,2025-03-14,10.340000,BUY,44.153764,N/A,"(25, 35)",Technology,N/A,3983635200,N/A,14.57143,None,None,None
5,600221.SS,2025-03-14,1.510000,BUY,34.228187,38.0,"(15, 25)",Industrials,N/A,65845030912,0.125,NaN,None,None,None
6,300169.SZ,2025-03-14,7.950000,BUY,41.734417,N/A,"(N/A, N/A)",Basic Materials,N/A,2549195008,N/A,NaN,None,None,None


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

YF.download() has changed argument auto_adjust default to True



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

Warnings: {}
Error fetching data for ALTM: list index out of range
NKE: Earnings on 2025-03-20 00:00:00
PDD: Earnings on 2025-03-20 00:00:00


In [7]:
every_day_printer(date = datetime.today().strftime("%Y-%m-%d"))

['AAPL', 'AAL', 'ASML', 'AMD', 'ABNB', 'ALK', 'ALB', 'ALTM', 'ARKK', 'ARM', 'AMC', 'ANET', 'BABA', 'BAC', 'BILI', 'BNTX', 'C', 'CDNS', 'CVNA', 'COF', 'COIN', 'COST', 'CRM', 'DASH', 'DAL', 'DDOG', 'DIS', 'DUOL', 'GOOGL', 'HOOD', 'ISRG', 'JPM', 'KO', 'KRE', 'LYFT', 'LLY', 'MDB', 'META', 'MSFT', 'NKE', 'NEE', 'NVDA', 'NVO', 'ORCL', 'OXY', 'PANW', 'PDD', 'PLTR', 'PFE', 'RBLX', 'SHOP', 'SNOW', 'SOFI', 'SMCI', 'SNAP', 'SBUX', 'SPOT', 'SOUN', 'INTC', 'BA', 'TMF', 'TRV', 'TSLA', 'TSM', 'UAL', 'UBER', 'UHAL', 'UPS', 'WFC', 'XOM', 'DHI', 'MMM', 'NFLX', '601288.SS', '688091.SS', '600221.SS', '600733.SS', '300169.SZ', '603628.SS', '601808.SS', 'BTC-USD', 'ETH-USD']
Today is 2025-03-17


,Company Code,Date,Daily Price,Recommendation,60 DAY RSI,P/E Ratio,Recommended PE,Category,Dividend Yield,Market Cap,Earnings Growth,One Year Target,Analyst Buy,Analyst Hold,Analyst Sell
0,300169.SZ,2025-03-17,7.820000,BUY,40.181820,N/A,"(N/A, N/A)",Basic Materials,N/A,2549195008,N/A,None,None,None,None
1,BTC-USD,2025-03-17,83811.289062,BUY,43.103788,N/A,"(N/A, N/A)",N/A,N/A,1662656184320,N/A,None,None,None,None


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Warnings: {}
Error fetching data for ALTM: list index out of range
NKE: Earnings on 2025-03-20 00:00:00
PDD: Earnings on 2025-03-20 00:00:00
601808.SS: Earnings on 2025-03-25 00:00:00
